<img src="https://brand.umich.edu/assets/brand/style-guide/logo-guidelines/U-M_Logo-Horizontal-Hex.png" alt="Drawing" style="width: 300px;" align="left"/><br>
    
## Week 4: Dashboard

For this notebook, you are building a prototype dashboard. The specific user persona we have in mind for you to design around is a non-technical instructor who is relatively new to teaching online and has moderate to low data literacy skills. 

The instructor, as we have mentioned in previous Notebooks, is looking to run a report about 25% of the way through the course in order to identify who he or she should do a formal check-in with.
 
To help you in building your dashboard, this notebook provides a brief introduction to the [jupyter-dash extension](https://github.com/plotly/jupyter-dash) for building a [Plotly Dash](https://github.com/plotly/dash) app within Jupyter environments. You can check out the documentation and some tutorials for Plotly Dash [here](https://dash.plotly.com/).

Resources:
* [A medium blog - Introducing JupyterDash](https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e)
* [Plotly Dash documentation](https://dash.plotly.com/)
* [A Youtube tutorial on dash](https://www.youtube.com/watch?v=hSPmj7mK6ng)
* [Dash gallery](https://dash-gallery.plotly.host/Portal/)

You can also check out [OU Analyse](https://analyse.kmi.open.ac.uk/) as an example of how the authors of the dataset have developed a dashboard for their institution. You can request a demo of the dashboard by entering your email.

In [24]:
# !pip install jupyter-dash  ## should already be installed

import pandas as pd
import numpy as np

# Import jupyter dash
from jupyter_dash import JupyterDash
import os
try:
    os.environ.pop('http_proxy')
    os.environ.pop('https_proxy')
except KeyError:
    pass

# Import dash
import dash
from dash import dcc
from dash import html

# Import plotly
import plotly.graph_objs as go
import plotly.express as px

# Set up jupyter proxy
JupyterDash.infer_jupyter_proxy_config()

In [25]:
### Additional imports
# from typing import  List

In [26]:
### Additional imports
from typing import  List


#### Use Boostrap layout to simplify organization

`Note`
- Ignore if this error occurs, the import will and should succeed
`ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash 2.4.1 requires dash-core-components==2.0.0, which is not installed.`

In [27]:
!pip install dash-html-components
!pip install dash-table
!pip install dash-core-components

In [28]:

import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from plotly.subplots import make_subplots
from dash import Dash, dash_table

### Read data

In [29]:
lr = pd.read_csv('assets/learning_resources.csv')
qt = pd.read_csv('assets/quizzes_tests.csv')
si = pd.read_csv('assets/student_info_pred.csv')
df = pd.read_csv('assets/country_indicators.csv')

#####  Items of interest
1. Instructor will need explanations and annotations
2. Date range limited to 25% of the course


In [30]:
si.sample()

,pred_fail,id_student,gender,highest_education,disability,final_result
605,0.18487,556082,M,High School + Advanced Placement,N,Pass


In [31]:
si['pred_pass'] = 1 - si['pred_fail']


#subset dataset to those with due_date <= 60 , followed by subsetting dataset where date_submitted <= 60
def qt_feature_engineer(qt:pd.DataFrame, num_analysis_days:int = 60):
    """
        This function adds a continuous feature and several caegorical features for each quiz and text that was conducted
        in the 60 day period of analysis.
    """
    #This subsets the dataframe to 2 quizzes and tests
    qt_60 = qt[qt['due_date'] <= num_analysis_days].copy()
    qt_60 = qt_60[(qt_60['date_submitted'].isna()) | (qt_60['date_submitted'] <= num_analysis_days)].copy()

    #create a continuous weighted score
    qt_60['weighted_score'] = (qt_60['weight'] * qt_60['score'] ) / 100
    
    #Create a categorical column for whether submitted (1) or not (0) and if submitted,
    qt_60['submitted'] = np.where(qt_60['date_submitted'].isna(), "Not Submitted", "Submitted")
    #Create a categorical column for delay (1) or not (0) based on date_submitted <= due_date
    qt_60['submission_on_time'] = np.where(qt_60['date_submitted'] <= qt_60['due_date'], "On-Time Submission", "Delayed Submission")
    
    #Features : pivot for weighted score in each quiz and test 
    qt_wide = qt_60[['id_student', 'assignment_name', 'weighted_score', 'submitted', 'submission_on_time']].pivot(index=['id_student'],columns=['assignment_name'] , values=['weighted_score', 'submitted', 'submission_on_time'])
    #The pivot create multi-level column heading, drop the score level
    qt_wide = qt_wide.reset_index()
    #The weighted scoes, submitted and submission_on_time can all be set to 0 if the subset does not 
    #contain information for the student
    qt_wide = qt_wide.fillna(0)
    qt_wide.columns = qt_wide.columns.map('|'.join).str.strip('|')
   
    return qt_60, qt_wide


#subset dataset to those with date <= 60 
def lr_feature_engineer(lr:pd.DataFrame, num_analysis_days:int = 60):
    """
        This function adds a continuous feature and several caegorical features for each quiz and text that was conducted
        in the 60 day period of analysis.
    """
    #This subsets the dataframe to 812 students The one lost has no activity in 60 days :472949

    lr_60 = lr[lr['date'] <= num_analysis_days].copy()
    #I choose to sum the clicks for various specific ids within an activity_type together
    lr_wide = lr_60.groupby(['id_student', 'activity_type']).agg(sum_click=('sum_click', 'sum')).reset_index()

    lr_wide = lr_wide[['id_student', 'activity_type', 'sum_click']].pivot(index=['id_student'],columns=['activity_type'] , values=['sum_click'])
    #The pivot create multi-level column heading
    lr_wide  = lr_wide.fillna(0)
    lr_wide.columns = lr_wide.columns.droplevel(0)
    lr_wide.reset_index(inplace=True)
    lr_wide = lr_wide.rename_axis(None, axis=1)
    lr_avg = lr_wide.set_index('id_student').mean().reset_index().rename(columns={0:"click_mean", 'index':'activity_type'})
    lr_avg['click_mean'] =  np.round(lr_avg['click_mean'], 0)
    lr_60_student = lr_60.groupby(['id_student', 'activity_type'])['sum_click'].sum().reset_index()
    #Merge to create a column with mean alongside the sum of clicks
    lr_60_student = lr_60_student.merge(lr_avg, how='left', left_on='activity_type', right_on='activity_type')

    return lr_60, lr_60_student

#### Limit the dataframes to the 25% timeframe of analysis

In [32]:
qt_60 , qt_wide = qt_feature_engineer(qt, num_analysis_days=60)
print(qt_60.shape)
qt_60.sample(3)

(3215, 9)


,id_student,assignment_name,due_date,weight,date_submitted,score,weighted_score,submitted,submission_on_time
8694,556785,Quiz 1,23,2.0,25.0,90,1.80,Submitted,Delayed Submission
8680,556780,Quiz 1,23,2.0,25.0,90,1.80,Submitted,Delayed Submission
6524,540298,Quiz 1,23,2.0,25.0,57,1.14,Submitted,Delayed Submission


In [33]:
lr_60, lr_60_student = lr_feature_engineer(lr, num_analysis_days= 60)
print(lr_60.shape)
lr_60_student.sample(3)

(193978, 5)


,id_student,activity_type,sum_click,click_mean
1892,524378,wiki,52,34.0
609,323146,resource,20,78.0
1093,423757,forum,46,152.0


### Helper Functions

In [34]:
label_blue = "#163057"
slider_color =  '#77b0b1'
def get_student_id_list(si:pd.DataFrame, option_all:bool=False) -> List:
    """
        This function returns a dictionary of student id label and student id value pairs.
        If option_all is set then 
    
    """
    # Step 1
    student_list = list(si['id_student'])
    # Create a checklist based on assignment_name
    options=[{'label': id, 'value': id} for id in student_list]
    if option_all:
        options.append({"label":"Select All", "value":"all"})
    return options

def get_studentid_filter(options:List, multi:bool=False, dropdown_id:str='student_selector'):
    """
        This fuunction returns an html div cotaining a dropdown with student ids
        input:options: List of dictionary of student id label and student id vaue pairs
        input:multi:Indicates whether the dropdown list should be multiselect
        input:dropdown_id: id for created dropdown
        output: dropdown div with id 'student_selector' for the dropdown selected
        
    """
        
    student_id_dropdown = html.Div(children=[
                                            html.Label(children = " Select Student:",
                                                   style={
                                                        "font-weight": "bold",
                                                        "color": slider_color,
                                                    },
                                            ),

                                            dcc.Dropdown(
                                                    id='student_selector',
                                                    options=options,
                                                    value=options[0]['value'],
                                                    multi=multi
                                                ),

                            ])
    return student_id_dropdown


def get_prob_slider_div(min_slider_range:int=0, max_slider_range:int=1, step:float=0.1):
    """
        This function creates an html div containing a slider with specified range. It has a feedback for selected range.
    """
    prob_slider_div = (
            html.Div(children=[

                          html.Label(children = "Select Range of Predicted Fail Probablities:",
                                       style={
                                            "font-weight": "bold",
                                            "color": slider_color,
                                        },
                                      ),
                
                            dcc.RangeSlider(
                                    id='predprob_range_slider',
                                    min=min_slider_range, max=max_slider_range, step=step,
                                    value=[min_slider_range, max_slider_range],
                                    marks={
                                        0: {'label':'0%', 'style': {'color': slider_color}},
                                        0.1:{'label' : '10%', 'style': {'color': slider_color}},
                                        0.2: {'label':'20%', 'style': {'color': slider_color}},
                                        0.3: {'label' :'30%', 'style': {'color': slider_color}},
                                        0.4: {'label' : '40%', 'style': {'color': slider_color}},
                                        0.5: {'label':'50%', 'style': {'color': slider_color}},
                                        0.6: {'label':'60%', 'style': {'color': slider_color}},
                                        0.7: {'label':'70%', 'style': {'color': slider_color}},
                                        0.8: {'label':'80%', 'style': {'color': slider_color}},
                                        0.9: {'label':'90%', 'style': {'color': slider_color}},
                                        1.0: {'label':'100%', 'style': {'color': slider_color}},
                                       
                                    },
                                ),
                            html.Div(id='output-container-range-slider',
                                     style={
                                            "color": slider_color,
                                            },
                                    )

        ])
    )
    return prob_slider_div



def get_card(text_str, card_id, card_size:str="H6"):
    """
        This function creates a card with cardbody containing some centered text
    """
    if card_size == "H6":
            header_card = dbc.Card(
                                [ dbc.CardBody(
                                    html.H6(text_str,className='text-center', id=card_id)
                                    )
                                ], 
                                color='dark',
                                inverse=True,
                                outline=False,
                                style={'height':'10vh'}
            )
    else:
         header_card = dbc.Card(
                                [ dbc.CardBody(
                                    html.P(text_str, className='text-center', id=card_id, style={"fontSize":10 }),  
                                    )
                                ], 
                                color='dark',
                                inverse=True,
                                outline=False,
                                style={'height':'8vh'}
                                 )
    return header_card



def get_student_id_display(card_id):
    """
        This function creates a row of student id card
    """
    return html.Div(# Header
            dbc.Row([
                        dbc.Col(get_card("Student ID: ", card_id, "P")),
                    ],
            justify='center'
            )
    )



####################### CSS formatting #############################################################
dashboard_colors = {
    "superdark-blue": "rgb(31,38,42)",
    "acid-pink": "rgb(213, 3, 160)",
    "black": "rgb(0, 0, 0)",
    "medium-blue-grey": "rgb(59,139,235)",
    "dark-blue": "rgb(0,0,0)",
    "light-green": "rgb(245,252,255)",  # This is filter background
    "white": "rgb(251, 251, 252)",
    "light-grey": "rgb(208, 206, 206)",
    "superlight-blue": "rgb(242,242,242)",
    "label_blue" : "#163057",
    "slider_color" :  '#77b0b1'
}

## these are divs that set the look of a row and a column
externalgraph_rowstyling = {"margin-left": "15px", "margin-right": "15px"}

externalgraph_colstyling = {
    "border-radius": "10px",
    "border-style": "solid",
    "border-width": "1px",
    "border-color": dashboard_colors["superlight-blue"],
    "background-color": dashboard_colors["superlight-blue"],
    "box-shadow": "0px 0px 17px 0px rgba(186, 218, 212, .5)",
    "padding-top": "10px",
}


# Filter Styling
filterdiv_borderstyling = {
    "border-radius": "0px 0px 10px 10px",
    "border-style": "solid",
    "border-width": "1px",
    "border-color": dashboard_colors["light-green"],
    "background-color": dashboard_colors["light-green"],
    "box-shadow": "2px 5px 5px 1px rgba(255, 101, 131, .5)",
}

####################### Uniform graphing elements

dashboard_title = {"font": {"size": 16, "color": dashboard_colors["white"]}}

dashboard_xaxis = {
    "showgrid": False,
    "linecolor": dashboard_colors["superdark-blue"],
    "color": dashboard_colors["superdark-blue"],
    "tickangle": 315,
    "titlefont": {"size": 12, "color": dashboard_colors["black"]},
    "tickfont": {"size": 11, "color": dashboard_colors["black"]},
    "zeroline": False,
}

dashboard_yaxis = {
    "showgrid": True,
    "color": dashboard_colors["superdark-blue"],
    "gridwidth": 0.5,
    "gridcolor": dashboard_colors["superdark-blue"],
    "linecolor": dashboard_colors["superdark-blue"],
    "titlefont": {"size": 12, "color": dashboard_colors["superdark-blue"]},
    "tickfont": {"size": 11, "color": dashboard_colors["superdark-blue"]},
    "zeroline": False,
}

dashboard_font_family = "Dosis"

dashboard_legend = {
    "orientation": "h",
    "yanchor": "bottom",
    "y": 1.01,
    "xanchor": "right",
    "x": 1.05,
    "font": {"size": 9, "color": dashboard_colors["superdark-blue"]},
}  # Legend will be on the top right, above the graph, horizontally

dashboard_margins = {
    "l": 5,
    "r": 5,
    "t": 45,
    "b": 15,
}  # Set top margin to in case there is a legend

dashboard_layout = go.Layout(
    font={"family": dashboard_font_family},
    title=dashboard_title,
    title_x=0.5,  # Align chart title to center
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    xaxis=dashboard_xaxis,
    yaxis=dashboard_yaxis,
    height=270,
    legend=dashboard_legend,
    margin=dashboard_margins,
)

def data_bars_diverging(df, column, color_above=dashboard_colors['acid-pink'], color_below=dashboard_colors['slider_color']):
    '''
        This function creates a diverging data bar in each row of a column in a dataframe being displayed.
        The threshold is the mid point = 50% failure prediction
    '''
    n_bins = 100
    bounds = [i * (1.0 / n_bins) for i in range(n_bins + 1)]
    col_max = df[column].max()
    col_min = df[column].min()
    ranges = [
        ((col_max - col_min) * i) + col_min
        for i in bounds
    ]
    midpoint = (col_max + col_min) / 2.

    styles = []
    for i in range(1, len(bounds)):
        min_bound = ranges[i - 1]
        max_bound = ranges[i]
        min_bound_percentage = bounds[i - 1] * 100
        max_bound_percentage = bounds[i] * 100

        style = {
            'if': {
                'filter_query': (
                    '{{{column}}} >= {min_bound}' +
                    (' && {{{column}}} < {max_bound}' if (i < len(bounds) - 1) else '')
                ).format(column=column, min_bound=min_bound, max_bound=max_bound),
                'column_id': column
            },
            'paddingBottom': 2,
            'paddingTop': 2
        }
        if max_bound > midpoint:
            background = (
                """
                    linear-gradient(90deg,
                    white 0%,
                    white 50%,
                    {color_above} 50%,
                    {color_above} {max_bound_percentage}%,
                    white {max_bound_percentage}%,
                    white 100%)
                """.format(
                    max_bound_percentage=max_bound_percentage,
                    color_above=color_above
                )
            )
        else:
            background = (
                """
                    linear-gradient(90deg,
                    white 0%,
                    white {min_bound_percentage}%,
                    {color_below} {min_bound_percentage}%,
                    {color_below} 50%,
                    white 50%,
                    white 100%)
                """.format(
                    min_bound_percentage=min_bound_percentage,
                    color_below=color_below
                )
            )
        style['background'] = background
        styles.append(style)

    return styles



def data_bars(df, column):
    """
        This function creates databars within a data table in dash plotly
    """
    n_bins = 100
    bounds = [i * (1.0 / n_bins) for i in range(n_bins + 1)]
    ranges = [
        ((df[column].max() - df[column].min()) * i) + df[column].min()
        for i in bounds
    ]
    styles = []
    for i in range(1, len(bounds)):
        min_bound = ranges[i - 1]
        max_bound = ranges[i]
        max_bound_percentage = bounds[i] * 100
        styles.append({
            'if': {
                'filter_query': (
                    '{{{column}}} >= {min_bound}' +
                    (' && {{{column}}} < {max_bound}' if (i < len(bounds) - 1) else '')
                ).format(column=column, min_bound=min_bound, max_bound=max_bound),
                'column_id': column
            },
            'background': (
                """
                    linear-gradient(90deg,
                    #0074D9 0%,
                    #0074D9 {max_bound_percentage}%,
                    white {max_bound_percentage}%,
                    white 100%)
                """.format(max_bound_percentage=max_bound_percentage)
            ),
            'paddingBottom': 2,
            'paddingTop': 2
        })

    return styles



def get_pred_fail_table(table_id:str, df:pd.DataFrame):
    """
        This function creates a plotly data table with the prediction of failure charted as a data bar
    """
    PAGE_SIZE = 20
    
    df['pred_fail'] =   np.round( df['pred_fail'] * 100, 2)
    df = df.sort_values(by=['pred_fail'], ascending=False)
    pred_fail_table = dash_table.DataTable(
        id = table_id,
        data=df.to_dict('records'),
      
        columns=[{'name': i.replace("pred_fail", "Failure Probability").replace("_", ' ').capitalize(), 'id': i} for i in df.columns],
        style_data_conditional=(
            data_bars_diverging(df, 'pred_fail') 
        ),
        style_cell={
            'width': '100px',
            'minWidth': '100px',
            'maxWidth': '100px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
        },
        column_selectable='single',
        #sort_action='native',
        sort_action='none',
        page_current=0,
        page_size=PAGE_SIZE,
      
    )
    return pred_fail_table

def getCellStudentId(active_cell, page_current, page_size, data):
    if active_cell:
        col = active_cell['column_id']
        row = active_cell['row']
        cellData = data[page_current * page_size + row][col]
        if col == 'id_student':
            return cellData
    return options[0]['value']


# A quick tutorial

In essence, a plotly-dash dashboard consists of 3 components:
* The **dash components** (e.g., dropdown, slider, checklist, etc.). See the documentation [here](https://dash.plotly.com/dash-core-components)
* The **plotly** graphs (e.g., linegraph, scatter plot, heatmap, etc.). See the documentation [here](https://plotly.com/python/)
* The **callback** to connects the dash components to plotly graphs, making it an interactive dashboard. See the documentation [here](https://dash.plotly.com/basic-callbacks) 

## Step 1. The Dash components (i.e. layouts)

In [35]:
score = qt.groupby('assignment_name')['score'].agg({np.mean,np.median, np.std}).reset_index(drop = False)

In [ ]:
app = JupyterDash(__name__)

# Create server variable
server = app.server

# Create a unique list of code_module
available_indicators1 = score['assignment_name'].unique()

# Step 1

app.layout = html.Div([
    
            # Create a html title for the dashboard
            html.H1("This is the title"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators1],
                value='Quiz 1' # the default assignment
            )])

# Run the app
app.run_server(mode="inline", port = 8100)

# You will see we have the dropdown menu but nothing happens yet

## Step 2 & 3. Callback and create plotly graph

In [ ]:
# Create a unique list of code_module
assess_list = score['assignment_name'].unique()

# Step 1
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is a bar chart"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in assess_list],
                value='Quiz 1' # the default
            )])

# Step 2
# Callback using input from dropdown menu to generate graph
# You can have multiple inputs and multiple outputs
@app.callback(
    dash.dependencies.Output('graph-with-dropdown', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(assignment_name):
    filtered_score = score[(score.assignment_name == assignment_name)]
    figure = px.bar(filtered_score, x = 'assignment_name', y = 'mean')
    return figure # You must return all the output(s) in step 2
    
# Run the app
app.run_server(mode = "inline", port = 8101)

In [ ]:
# Create a unique list of assignment_name
assess_list = score['assignment_name'].unique()

# Step 1
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is a scatter plot"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph'),
    
            # Create a checklist based on assignment_name
            dcc.Checklist(
                    id = 'checklist',
                    options=[{'label': i, 'value': i} for i in assess_list],
                    value=assess_list # Default values contain all assignment_name
            )
])

# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('checklist', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(assignment_name):
    figure = px.scatter(score, 
                     x="mean", 
                     y="std", 
                     color="assignment_name",
                     hover_name='assignment_name')
    return figure # You must return all the output(s) in step 2
    
# Run the app
app.run_server(mode="inline", port = 8102)

# Building Your Dashboard (40 points)

Your final product will need to have the following capabilities:

1. (10 points) View multiple students' predicted probabilities of failing the course
2. (10 points) View a student's quiz/test performances alongside a meaningful reference like a course average for a given quiz/test
3. (10 points) View a student's learning resource use alongside a meaningful reference like a course average for a given resource
4. (10 points) Intergrate the above three capabilties into a single, functional dashboard

This notebook is broken out into four sections, three to build individual components and one to integrate them. The final Integrated Dashboard cell will be graded and must include all three specified capabilities. If it does not, points will be awarded based on progress made in each Component cell.

There is a new column in `student_info.csv`, `fail_pred,` that represents the predicted probability of failure for each student from a baseline model.

**Note:** All quizzes, tests, and resources should only include information from on or before day 60 in the course.

### Dictionary
- **student_info.csv**
    - **id_student** = numeric; unique identifier for each student in the course
    - **gender** = character; M = "male", F = "female"
    - **highest_education** = character; “Some Graduate”, “Some Higher Education”, “High School + Advanced Placement”, “High School”, “No Formal Quals” (Categories ordered from highest documented education level attained to lowest documented education level attained)
    - **disability** = character; Y = "yes", N = "no"
    - **final_result** = character; "Fail", "Pass"
    - **fail_pred** = numeric; predicted probability from sample model


- **quizzes_tests.csv**
    - **id_student** = numeric; unique identifier for each student in the course
    - **assignment_name** = character; name of graded assignment (Quiz 1-7, Test 1-6, Final Exam)
    - **due_date** = numeric; date assignment was due (indexed as count in days from start of course, i.e., day 0)
    - **weight** = numeric; weight multiplied by score when generating final grade (weight * score / 100)
    - **date_submitted** = numeric; date student submitted assigned (indexed as count in days from start of course, i.e., day 0, NaN means students did not submit assignment)
    - **score** = numeric; score student earned on assignment (0 means students did not submit assignment)


- **learning_resources.csv**
    - **id_student** = numeric; unique identifier for each student in the course
    - **activity_type** = character; overarching label for learning activity students can access (“course_homepage”, “course_page”, “forum”, ‘resource”, “wiki”)
    - **activity_id** = numeric; unique identifier for specific learning activity student accessed within overacting `activity_type`
    - **date** = numeric; date student accessed specific `acitivity_id`  (indexed as count in days from start of course, i.e., day 0)
    - **sum_click** = numeric; count of clicks for `activity_id` on date

## Component 1
1. (10 points) View multiple students' predicted probabilities of failing the course

In [ ]:
app = JupyterDash(__name__)
server = app.server


# Step 1: Create a layout (title, dropdown menu, slider, etc...)

# Step 2: Callback to connect input(s) to output(s)

# Step 3: Define the graph with plotly express

    
# Un-comment to run the app
# app.run_server(mode="inline", port = 8052)

 ##  Component 1 : A two part figure (scroll down to see a scatter plot beneath the datatable)
 
 The provided website for Open University shows all students in a data table with the predicted failure probability
 
 I have created this components as a two piece figure combining the OU's data table with a scatter plot where a range of probabilities can be chosen to isolate a student predicted to fail within that range. The range can be chosen with the help of a slider whise range can be limited from either end of the slider.

In [ ]:
#https://dash.plotly.com/datatable/conditional-formatting?_gl=1*dxelux*_ga*MTY3ODgzNDY3LjE2NjQyOTc3NTA.*_ga_6G7EE0JNSC*MTY2NTU5MTgxMi4yOC4xLjE2NjU1OTM4NTAuMC4wLjA.

app = JupyterDash(__name__)

# Create server variable
server = app.server

# Step 1
options = get_student_id_list(si, True)

#Create slider for probability range
min_slider_range = 0
max_slider_range = 1


app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("Predicted Fail Probability"),
            html.H5("Grades and Activity For First Quarter"),
            
            html.Div(
                
                    children = [
                                 #Create a paged table of students and theor predicted probability of failure   
                                 get_pred_fail_table("pred_table_1", si[['id_student', 'pred_fail','gender', 'disability', 'highest_education']]),
                                 # Create a graph, we will configure the graph using plotly express in step 3 
                                 dcc.Graph(id='student_fail_prob'),
                                ]
            
            ),
            get_prob_slider_div(),
 
])


##########DEFINE CALLBACKS
# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('student_fail_prob', 'figure'),
    [dash.dependencies.Input('predprob_range_slider', 'value')]
)

# Step 3
# Define the graph with plotly express
def update_figure(prob_slider_range):
    df = si.copy()
    df['student_selected'] = np.where((df['pred_fail'] >= prob_slider_range[0])
                                         & ( df['pred_fail'] <= prob_slider_range[1]),
                                      'selected',
                                      'not_selected'
                             )
                                         
    df['size'] = np.where((df['pred_fail'] >= prob_slider_range[0])
                                         & ( df['pred_fail'] <= prob_slider_range[1]),
                                      10,
                                      2
                             )
        
    figure = px.scatter(df, 
                     x="id_student", 
                     y="pred_fail",
                     #size="size", 
                     size_max=10,
                     color="student_selected",
                     color_discrete_map={"selected":slider_color,"not_selected" : 'lightgrey'},
                     symbol='student_selected',
                     labels={
                         "id_student": "Student Id",
                         "pred_fail": "Predicted Probability Of Fail"
                     },
                     hover_name='id_student',
                     hover_data={'student_selected':False, # remove species from hover data
                             'id_student':':.0f', # customize hover for column of y attribute
                             'pred_fail':True, # add other column, default formatting
                                }
                )
    
    #Add rectangle to follow range
    figure.add_shape(type="rect",
                    x0=0, y0=prob_slider_range[0], x1=figure.data[0].x.max(), y1=prob_slider_range[1],
                    line=dict(
                        color="RoyalBlue",
                        width=2,
                    ),
                    fillcolor=slider_color,
                    opacity=0.3
                )
    
    figure.update_layout(showlegend=False) 
    figure.update_layout(yaxis_tickformat='.0%')
    figure.update_layout( xaxis = dict(
                            tickmode = 'array',
                            tickvals = [0, 500000, 1000000, 1500000, 2000000, 2500000],
                            ticktext = ['0', '50000', '1000000', '1500000', '2000000', '2500000']
                        )
                        )
    
    return figure # You must return all the output(s) in step 2

@app.callback(
        dash.dependencies.Output('output-container-range-slider', 'children'),
       [dash.dependencies.Input('predprob_range_slider', 'value')]
)
def update_output(prob_slider_range):
    return f'Selected Range: {prob_slider_range[0] * 100}% - {prob_slider_range [1]*100}%'


# Run the app
app.run_server(mode="inline", port = 8100)


## Component 2

2. (10 points) View a student's quiz/test performances alongside a meaningful reference like a course average for a given quiz/test

**Note:** All quizzes, tests, and resources should be from on or before day 60 in the course.

In [ ]:
# But the concept is different. In qcut, when you pass q=4, it will try to divide the population equally and calculate
#the bin edges accordingly. But in the cut method, 
#it divides the range of the data in equal 4 and the population will follow accordingly.
#qt_bins = pd.cut(qt_60[qt_60['assignment_name'] == 'Test 1' ]['score'], bins=3,  precision=0)
#qt_bins = pd.DataFrame(qt_bins.value_counts()).reset_index()
#0-33, 34-66, 67-100

In [ ]:
### Student grades plot
# Create a checklist based on assignment_name
options = get_student_id_list(si)
qt_avg = qt_60[['assignment_name', 'score']].groupby(['assignment_name']).mean().reset_index()


app = JupyterDash(__name__)
server = app.server



app.layout = html.Div([
            # Create a html title for the dashboard
            html.H3("Student Assignment Grades"),
            html.H5("Grades For First Quarter"),
            
            html.Div(
                    children = [
                                get_student_id_display("student_card_id"),
                                # Create a graph, we will configure the graph using plotly express in step 3 
                                dcc.Graph(id='student_assignment_grades'),
                                ]
            
            ),
            get_studentid_filter(options)
 
])

# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('student_assignment_grades', 'figure'),
    [dash.dependencies.Input('student_selector', 'value')]
)

# Step 3
# Define the graph with plotly express
def update_figure(id_student):
    #Get the student data
    df = qt_60[qt_60['id_student'] == id_student].copy()
    
    fig = go.Figure()

    student_quiz1_score = df[df['assignment_name'] == 'Quiz 1']['score'].values[0]
    quiz1_reference_avg = qt_avg[qt_avg['assignment_name'] == 'Quiz 1']['score'].values[0]
    quiz1_threshold_avg = qt_avg[qt_avg['assignment_name'] == 'Quiz 1']['score'].values[0]

    student_quiz2_score = df[df['assignment_name'] == 'Quiz 2']['score'].values[0]
    quiz2_reference_avg = qt_avg[qt_avg['assignment_name'] == 'Quiz 2']['score'].values[0]
    quiz2_threshold_avg = qt_avg[qt_avg['assignment_name'] == 'Quiz 2']['score'].values[0]

    student_test1_score = df[df['assignment_name'] == 'Test 1']['score'].values[0]
    test1_reference_avg = qt_avg[qt_avg['assignment_name'] == 'Test 1']['score'].values[0]
    test1_threshold_avg = qt_avg[qt_avg['assignment_name'] == 'Test 1']['score'].values[0]

    student_test2_score = df[df['assignment_name'] == 'Test 2']['score'].values[0]
    test2_reference_avg = qt_avg[qt_avg['assignment_name'] == 'Test 2']['score'].values[0]
    test2_threshold_avg = qt_avg[qt_avg['assignment_name'] == 'Test 2']['score'].values[0]


    fig.add_trace(go.Indicator(
        mode = "number+gauge+delta", value = student_quiz1_score,
        delta = {'reference': quiz1_reference_avg, 'position': "top"},
        domain = {'x': [0.5, 1], 'y': [0.10, 0.2]},
        title = {'text': "Quiz 1",  'font': {"size": 12}},
        gauge = {
            'shape': "bullet",
            'axis': {'range': [None, 100]},
            'threshold': {
                'line': {'color': slider_color, 'width': 3},
                'thickness': 0.5,
                'value': quiz1_reference_avg},
            'steps': [
                {'range': [0, 33], 'color': "gray"},
                {'range': [33, 66], 'color': "lightgray"}],
            'bar': {'color': label_blue}}))

    fig.add_trace(go.Indicator(
        mode = "number+gauge+delta", value = student_quiz2_score,
        delta = {'reference': quiz2_reference_avg, 'position': "top"},
        domain = {'x': [0.5, 1], 'y': [0.30, 0.40]},
        title = {'text': "Quiz 2",  'font': {"size": 12}},
        gauge = {
            'shape': "bullet",
            'axis': {'range': [None, 100]},
            'threshold': {
                'line': {'color': slider_color, 'width': 3},
                'thickness': 0.5,
                'value': quiz2_reference_avg},
            'steps': [
                {'range': [0, 33], 'color': "gray"},
                {'range': [33, 66], 'color': "lightgray"}],
            'bar': {'color': label_blue}}))

    fig.add_trace(go.Indicator(
        mode = "number+gauge+delta", value = student_test1_score,
        delta = {'reference': test1_reference_avg, 'position': "top"},
        domain = {'x': [0.5, 1], 'y': [0.5, 0.6]},
        title = {'text': "Test 1",  'font': {"size": 12}},
        gauge = {
            'shape': "bullet",
            'axis': {'range': [None, 100]},
            'threshold': {
                'line': {'color': slider_color, 'width': 3},
                'thickness': 0.5,
                'value': test1_reference_avg},
            'steps': [
                {'range': [0, 33], 'color': "gray"},
                {'range': [33, 66], 'color': "lightgray"}],
            'bar': {'color': label_blue}}))

    fig.add_trace(go.Indicator(
        mode = "number+gauge+delta", value = student_test2_score,
        delta = {'reference': test2_reference_avg, 'position': "top"},
        domain = {'x': [0.5, 1], 'y': [0.7, 0.8]},
        title = {'text': "Test 2",  'font': {"size": 12}},
        gauge = {
            'shape': "bullet",
            'axis': {'range': [None, 100]},
            'threshold': {
                'line': {'color': slider_color, 'width': 3},
                'thickness': 0.5,
                'value': test2_reference_avg},
            'steps': [
                {'range': [0, 33], 'color': "gray"},
                {'range': [33, 66], 'color': "lightgray"}],
            'bar': {'color': label_blue}}))

 
    fig.add_annotation(
                xref="x domain",
                yref="y domain",
                # The Text will be at 0 along x axis, starting from the left
                x=0.0,
                # The Text will be at 0 along the y axis, starting from the bottom
                y=0.0,
                showarrow=False,
                text="* Grey bars are grade ranges : 0-33, 34-66, 67-100.<br>* Middle blue bar denotes student grade.<br>* Short blue vertical line is the class average for the assignment.<br>* Grade differential to the average seen on right.",
                align='left',
                font_size=8
    )
    
  
    
    fig.update_layout(height = 300 ,
                      margin = {'t':0, 'b':0, 'l':0},
                     )


    return fig # You must return all the output(s) in step 2

## Callback for Student id card
@app.callback(
    dash.dependencies.Output('student_card_id','children'),
   
    [dash.dependencies.Input('student_selector', 'value')]
)

def update_cards(id_student):
    return f"Student ID:{id_student}"  


# Run the app
app.run_server(mode="inline", port = 8102)


## Component 3

3. (10 points) View a student's learning resource use alongside a meaningful reference like a course average for a given resource

**Note:** All quizzes, tests, and resources should be from on or before day 60 in the course.

In [ ]:
### Learning Resource Clicks
app = JupyterDash(__name__)
server = app.server

# Step 1
options = get_student_id_list(si)


app.layout = html.Div([
                        # Create a html title for the dashboard
                        html.H3("Learning Resource Access"),
                        html.H5("Activity For First Quarter"),

                        html.Div(

                                children = [
                                            get_student_id_display("student_card_id"),
                                            # Create a graph, we will configure the graph using plotly express in step 3 
                                            dcc.Graph(id='student_learning_resource_access'),
                                            ]

                                ),

                       get_studentid_filter(options)
        ])

# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('student_learning_resource_access', 'figure'),
    [dash.dependencies.Input('student_selector', 'value')]
)

# Step 3
# Define the graph with plotly express
def update_figure(id_student):
    df = lr_60_student[lr_60_student['id_student'] == id_student].copy()
    df = df.melt(id_vars=['id_student', 'activity_type'])
    

    fig = px.bar(df, y="activity_type", x="value",
                 text_auto=True,
                 color="variable", 
                 hover_name='id_student',
                 hover_data={
                         'variable':False,
                         'id_student':':.0f', # customize hover for column of y attribute
                         'value':True, # add other column, default formatting
                 },
                 labels={'activity_type':'Activity Type',
                         "value" : "Clicks",
                         "sum_click": "Total Clicks"
                        
                        },
                 color_discrete_map={
                        'sum_click' : '#071633',
                        'click_mean' : slider_color
                    },
                barmode = 'group')
   
    fig.update_layout(legend_title="Click Type")
    
    newnames = {'sum_click':'Student Clicks', 'click_mean': 'Click Average For Course'}
    fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
    
    return fig # You must return all the output(s) in step 2

## Callback for Student id card
@app.callback(
    dash.dependencies.Output('student_card_id','children'),
   
    [dash.dependencies.Input('student_selector', 'value')]
)

def update_cards(id_student):
    return f"Student ID:{id_student}"  

# Run the app
app.run_server(mode="inline", port = 8102)

---

## Integrated Dashboard

4. (10 points) Intergrate the above three capabilties into a single, functional dashboard

In [ ]:
# !pip install dash-html-components
# !pip install dash-table
# !pip install dash-core-components

In [38]:
####################################################################################################
# Import dash core components (dcc), html and bootstrap components
####################################################################################################
# !pip install jupyter-dash  ## should already be installed


#from app import app


app = JupyterDash(__name__)

# Create server variable
server = app.server



################################################################################################################################################## SET UP END
# Setup dataframes 
################################################################################################################################################## SET UP END

qt_avg = qt_60[['assignment_name', 'score']].groupby(['assignment_name']).mean().reset_index()


################################################################################################################################################## SET UP END
# Filter  data
################################################################################################################################################## SET UP END

score = qt.groupby('assignment_name')['score'].agg({np.mean,np.median, np.std}).reset_index(drop = False)
# Create a unique list of code_module
available_indicators1 = score['assignment_name'].unique()

options = get_student_id_list(si)


####################################################################################################
# DEFINE REUSABLE COMPONENTS AS FUNCTIONS
####################################################################################################

# Header with logo
def get_header():

    header = html.Div(
        [
            html.Div(
                [
                    html.Img(
                        #src=app.get_asset_url("LogoForDashboard.png"),
                        src = 'assets/LogoForDashboard.png',
                        height="100%",
                        width="auto",
                    )
                ],
                className="col-1",
                style={"vertical-align": "middle"},
            ),  # Same as img width, allowing to have the title centrally aligned
            html.Div(
                [
                    html.H2(
                        children="Learning Analytics Dashboard",
                        style={
                            "textAlign": "center",
                            "font-family": "Luminari",
                            "font-weight": "bold",
                            "color": dashboard_colors["acid-pink"],
                        },
                    )
                ],
                className="col-6",
                style={"padding-top": "1%"},
            ),
            html.Div(
                [],
                className="col-5",
                style={"align-items": "center", "padding-top": "1%", "height": "auto"},
            ),
        ],
        className="row",
        style={"height": "2%", "background-color": dashboard_colors["superdark-blue"]},
    )

    return header


def get_emptycol(w="10px"):
    """This returns an empty column of a defined width"""

    emptycol = html.Div(
        [html.Div([html.Br()], className="col-2")],
        className="column",
        style={"width": w, 
               "margin": "10px"},
    )
    return emptycol


def get_emptyrow(h="10px"):
    """This returns an empty row of a defined height"""

    emptyrow = html.Div(
        [html.Div([html.Br()], className="col-12")],
        className="row",
        style={"height": h},
    )
    return emptyrow



#####################
# Common Filter Navbar
def get_filterbar():
    filterNavbar = html.Div(
        [  # This should start an array of  four rows
            # create four rows for each filter and its label.
            html.Div(  # Start First Row
                [
                    html.Div([], className="col-1",),
                ],
                className="row",
            ),  # End first row
            html.Div(  # Start second filter row
                [
                    html.Div(
                        [
                            get_studentid_filter(options)
                         
                        ],
                        className="col-12",
                    ),
                ],
                className="row",
            ),  # End second filter row
        ],  # This should end array of filter rows
        className="col-2",
        style=externalgraph_colstyling,
    )  # End of the two column filter div
    return filterNavbar



##########################################
# Row to be filled with KPI elements
def get_KPIrow():
    KPIrow = html.Div(
        [  # Internal row
        
            html.Div([
                        # Header
                        dbc.Row([
                            dbc.Col(get_card("Class Size: 831", "class_size_id")),
                            dbc.Col(get_card("Period: 60 days", "period_id" ), width=30),
                            
                        ],
                        justify='center'
                        )
                    ])

        ],
        className="row",
        style=externalgraph_rowstyling,
    )  # Internal row END KPIS row
    return KPIrow


##########################################
# Row to be filled with student id displayed as header to each individual student id chart
def get_StudentIDrow(student_card_id:str):
    student_id_row = html.Div(
        [  # Internal row
        
            html.Div([
                        # Header
                        dbc.Row([
                            dbc.Col(get_card("Student ID:", student_card_id, "P")),
                            
                        ],
                        justify='center',
                        style={"height": "1%"}
                        )
                    ])

        ],
        className="row",
        style=externalgraph_rowstyling,
    )  # Internal row END KPIS row
    return student_id_row


##########################################
# Row to be filled with student id displayed as header to each individual student id chart
def get_StudentSubmissionRow(student_card_id:str, delayed_submission_id:str, not_submitted_id:str):
    student_id_row = html.Div(
        [  # Internal row
        
            html.Div([
                        # Header
                        dbc.Row([
                            dbc.Col(get_card("Student ID:", student_card_id, "P")),
                            dbc.Col(get_card("Delayed:", delayed_submission_id, "P")),
                            dbc.Col(get_card("Not Submitted:", not_submitted_id, "P")),
                            
                        ],
                        justify='center',
                        style={"height": "1%"}
                        )
                    ])

        ],
        className="row",
        style=externalgraph_rowstyling,
    )  # Internal row END KPIS row
    return student_id_row

####################################################################################################
#  MAIN
####################################################################################################
student = html.Div(
    [
        #####################
        # Row 1 : Header
        get_header(),
        #####################
        get_emptyrow(h="10px"),
         # Row 2 :
        html.Div(
            [
                html.Div(  # Start of div with 2-col for filters and 10 col for graphs split
                    [
                        #get_filterbar(),
                        get_KPIrow(),
                        get_emptycol(),
                    ],  # End of div with 2-col and 10-col split
                    className="row",
                    style=externalgraph_rowstyling,  # External row
                ),  # External row
            
                html.Div(  # Separator empty row
                    [
                         get_emptyrow(h="20px"),
                    ],
                    className="row",
                    style=externalgraph_rowstyling,  # External row
                ),  # External row
             
                # Another div with rowstyling
                html.Div(  # Start of div containing rows of charts
                    [
                         #Div containing Student Grade
                        html.Div(
                            children = [
                                        html.Div(id = 'output_display'),
                                        html.H5(
                                            children="Fail Prediction Probabilities",
                                            style={
                                                "text-align": "left",
                                                "color": dashboard_colors["medium-blue-grey"],
                                                "font-weight": "bold",
                                            },
                                        ),
                                         get_pred_fail_table(
                                             "pred_table_2",
                                             si[['id_student', 'pred_fail', 'gender', 'disability', 'highest_education']]),
                                        get_emptyrow(h="2px"),
                                        # Create student grades chart
                                        dcc.Graph(id='student_fail_prob'),
                                        get_prob_slider_div(),

                                        ],
                                        className="col-12",
                                        style=externalgraph_colstyling,
                        ),
                        
                        html.Div(  # Add separation row
                            [
                                 get_emptyrow(h="10px"),
                            ],
                            className="row",
                            style=externalgraph_rowstyling,  # External row
                        ),  

                        #Div containing Student Grade
                        html.Div(
                            [
                                
                                get_StudentSubmissionRow("student_card_id_1","delayed_submission","not_submitted"),
                                get_emptyrow(h="5px"),
                                html.H5(
                                    children="Comparison of Student Grades And Course Averages",
                                    style={
                                        "text-align": "left",
                                        "color": dashboard_colors["medium-blue-grey"],
                                        "font-weight": "bold",
                                    },
                                ),
                                # Create student grades chart
                                dcc.Graph(id='student_assignment_grades'),
                                       
                            ],
                            className="col-12",
                            style=externalgraph_colstyling,
                        ),
                        
                        html.Div(  #   Add separation row
                            [
                                 get_emptyrow(h="10px"),
                            ],
                            className="row",
                            style=externalgraph_rowstyling,  # External row
                        ),  
                        
                        #Div containing Recource Activity
                        html.Div(
                            [
                                get_StudentIDrow("student_card_id_2"),
                                get_emptyrow(h="5px"),
                                html.H5(
                                    children="Clicks Per Resource Type",
                                    style={
                                        "text-align": "left",
                                        "color": dashboard_colors["medium-blue-grey"],
                                        "font-weight": "bold",
                                    },
                                ),
                                # Create a graph, we will configure the graph using plotly express in step 3 
                                dcc.Graph(id='student_learning_resource_access'),
                                       
                            ],
                            className="col-12",
                            style=externalgraph_colstyling,
                        ),
                       get_emptyrow(h="10px"),  
                    ],  # Internal row
                    className="row",
                    style=externalgraph_rowstyling,  # External row
                ),  # External row
                get_emptyrow(h="10px"),
            ]
        ),  # This ends the div Row 2

    ]
)



####################Chart 1 Callbacks
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('student_fail_prob', 'figure'),
    [dash.dependencies.Input('predprob_range_slider', 'value'),
     dash.dependencies.Input('pred_table_2', 'active_cell'),
     dash.dependencies.Input('pred_table_2', "page_current"),
     dash.dependencies.Input('pred_table_2', "page_size"),
    ],
    dash.dependencies.State('pred_table_2', 'data')
    
)

# Step 3
# Define the graph with plotly express
def update_figure(prob_slider_range, active_cell, page_current, page_size, data):
    df = si.copy()
    id_student = getCellStudentId(active_cell, page_current, page_size, data)
    #Select sets of studenst using probability
    df['student_selected'] = np.where((df['pred_fail'] >= prob_slider_range[0])
                                         & ( df['pred_fail'] <= prob_slider_range[1]),
                                      'selected',
                                      'not_selected'
                             )
    #Additionally highlight the student in the dropdown box selection
    df['student_selected'] = np.where(df['id_student'] == id_student, "indv_student_selected", df['student_selected'])
   
    #The size of the student selected in the dropdown is to be increased                                      
    df['size'] = np.where(df['student_selected'].isin(['selected', 'not_selected']),2,10)
                                      
    figure = px.scatter(df, 
                     x="id_student", 
                     y="pred_fail",
                     size="size",
                     size_max=10,
                     color="student_selected",
                     color_discrete_map={"selected":slider_color,"not_selected" : 'lightgrey', "indv_student_selected" : dashboard_colors["acid-pink"]},
                     symbol='student_selected',
                     labels={
                         "id_student": "Student Id",
                         "pred_fail": "Predicted Probability Of Fail"
                     },
                     hover_name='id_student',
                     hover_data={'student_selected':False, # remove species from hover data
                             'id_student':':.0f', # customize hover for column of y attribute
                             'pred_fail':True, # add other column, default formatting
                                }
                )
    
    
    figure.update_layout(showlegend=False) 
    figure.update_layout(yaxis_tickformat='.0%')
    figure.update_layout( xaxis = dict(
                            tickmode = 'array',
                            tickvals = [0, 500000, 1000000, 1500000, 2000000, 2500000],
                            ticktext = ['0', '50000', '1000000', '1500000', '2000000', '2500000']
                        )
                        )
    
    return figure # You must return all the output(s) in step 2

@app.callback(
        dash.dependencies.Output('output-container-range-slider', 'children'),
       [dash.dependencies.Input('predprob_range_slider', 'value')]
)
def update_output(prob_slider_range):
    return f'Selected Range: {prob_slider_range[0] * 100}% - {prob_slider_range [1]*100}%'




####################Chart 2 Callbacks   
# Learning Resources Graph
@app.callback(
    dash.dependencies.Output('student_learning_resource_access', 'figure'),
    [dash.dependencies.Input('pred_table_2', 'active_cell'),
      dash.dependencies.Input('pred_table_2', "page_current"),
      dash.dependencies.Input('pred_table_2', "page_size"),
    ],
    dash.dependencies.State('pred_table_2', 'data')
)

# Define the graph with plotly express
def update_figure(active_cell, page_current, page_size, data):
    id_student = getCellStudentId(active_cell, page_current, page_size, data)
    df = lr_60_student[lr_60_student['id_student'] == id_student].copy()
    df = df.melt(id_vars=['id_student', 'activity_type'])
    

    fig = px.bar(df, x="activity_type", y="value",
                 text_auto=True,
                 color="variable", 
                 hover_name='id_student',
                 hover_data={
                         'variable':False,
                         'id_student':':.0f', # customize hover for column of y attribute
                         'value':True, # add other column, default formatting
                 },
                 labels={'activity_type':'Activity Type',
                         "value" : "Clicks",
                         "sum_click": "Total Clicks"
                        
                        },
                 color_discrete_map={
                        'sum_click' : '#071633',
                        'click_mean' : slider_color
                    },
                barmode = 'group')
   
    # Place legend horizontally at the bottom of the chart and remove axis labels since the title of the chart explains the axes
    fig.update_layout(legend_title="Click Type", legend=dict(orientation='h'))
    fig.update_xaxes(tickangle= 0, title_text = "")
    fig.update_yaxes(tickangle= 0, title_text = "")
    
    
    newnames = {'sum_click':'Student Clicks', 'click_mean': 'Click Average For Course'}
    fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
    
    return fig 



####################Chart 3 Callbacks
# Student Grades
@app.callback(
    dash.dependencies.Output('student_assignment_grades', 'figure'),
    [dash.dependencies.Input('pred_table_2', 'active_cell'),
      dash.dependencies.Input('pred_table_2', "page_current"),
      dash.dependencies.Input('pred_table_2', "page_size"),
    ],
    dash.dependencies.State('pred_table_2', 'data')

)
# Define the graph with plotly express
def update_figure(active_cell, page_current, page_size, data):
    #Get the student data
    id_student = getCellStudentId(active_cell, page_current, page_size, data)
    df = qt_60[qt_60['id_student'] == id_student].copy()
    
    fig = go.Figure()

    student_quiz1_score = df[df['assignment_name'] == 'Quiz 1']['score'].values[0]
    quiz1_reference_avg = qt_avg[qt_avg['assignment_name'] == 'Quiz 1']['score'].values[0]
    quiz1_threshold_avg = qt_avg[qt_avg['assignment_name'] == 'Quiz 1']['score'].values[0]

    student_quiz2_score = df[df['assignment_name'] == 'Quiz 2']['score'].values[0]
    quiz2_reference_avg = qt_avg[qt_avg['assignment_name'] == 'Quiz 2']['score'].values[0]
    quiz2_threshold_avg = qt_avg[qt_avg['assignment_name'] == 'Quiz 2']['score'].values[0]

    student_test1_score = df[df['assignment_name'] == 'Test 1']['score'].values[0]
    test1_reference_avg = qt_avg[qt_avg['assignment_name'] == 'Test 1']['score'].values[0]
    test1_threshold_avg = qt_avg[qt_avg['assignment_name'] == 'Test 1']['score'].values[0]

    student_test2_score = df[df['assignment_name'] == 'Test 2']['score'].values[0]
    test2_reference_avg = qt_avg[qt_avg['assignment_name'] == 'Test 2']['score'].values[0]
    test2_threshold_avg = qt_avg[qt_avg['assignment_name'] == 'Test 2']['score'].values[0]


    # Create each bullet chart with grade differential to average to the right
    fig.add_trace(go.Indicator(
        mode = "number+gauge+delta", value = student_quiz1_score,
        delta = {'reference': quiz1_reference_avg, 'position': "top"},
        domain = {'x': [0.5, 1], 'y': [0.10, 0.2]},
        title = {'text': "Quiz 1",  'font': {"size": 12}},
        gauge = {
            'shape': "bullet",
            'axis': {'range': [None, 100]},
            'threshold': {
                'line': {'color': slider_color, 'width': 3},
                'thickness': 0.5,
                'value': quiz1_reference_avg},
            'steps': [
                {'range': [0, 33], 'color': "gray"},
                {'range': [33, 66], 'color': "lightgray"}],
            'bar': {'color': label_blue}}))

    fig.add_trace(go.Indicator(
        mode = "number+gauge+delta", value = student_quiz2_score,
        delta = {'reference': quiz2_reference_avg, 'position': "top"},
        domain = {'x': [0.5, 1], 'y': [0.30, 0.40]},
        title = {'text': "Quiz 2",  'font': {"size": 12}},
        gauge = {
            'shape': "bullet",
            'axis': {'range': [None, 100]},
            'threshold': {
                'line': {'color': slider_color, 'width': 3},
                'thickness': 0.5,
                'value': quiz2_reference_avg},
            'steps': [
                {'range': [0, 33], 'color': "gray"},
                {'range': [33, 66], 'color': "lightgray"}],
            'bar': {'color': label_blue}}))

    fig.add_trace(go.Indicator(
        mode = "number+gauge+delta", value = student_test1_score,
        delta = {'reference': test1_reference_avg, 'position': "top"},
        domain = {'x': [0.5, 1], 'y': [0.5, 0.6]},
        title = {'text': "Test 1",  'font': {"size": 12}},
        gauge = {
            'shape': "bullet",
            'axis': {'range': [None, 100]},
            'threshold': {
                'line': {'color': slider_color, 'width': 3},
                'thickness': 0.5,
                'value': test1_reference_avg},
            'steps': [
                {'range': [0, 33], 'color': "gray"},
                {'range': [33, 66], 'color': "lightgray"}],
            'bar': {'color': label_blue}}))

    fig.add_trace(go.Indicator(
        mode = "number+gauge+delta", value = student_test2_score,
        delta = {'reference': test2_reference_avg, 'position': "top"},
        domain = {'x': [0.5, 1], 'y': [0.7, 0.8]},
        title = {'text': "Test 2",  'font': {"size": 12}},
        gauge = {
            'shape': "bullet",
            'axis': {'range': [None, 100]},
            'threshold': {
                'line': {'color': slider_color, 'width': 3},
                'thickness': 0.5,
                'value': test2_reference_avg},
            'steps': [
                {'range': [0, 33], 'color': "gray"},
                {'range': [33, 66], 'color': "lightgray"}],
            'bar': {'color': label_blue}}))

 
    fig.add_annotation(
                xref="x domain",
                yref="y domain",
                # The Text will be at 0 along x axis, starting from the left
                x=0.0,
                # The Text will be at 0 along the y axis, starting from the bottom
                y=0.0,
                showarrow=False,
                text="* Grey bars are grade ranges : 0-33, 34-66, 67-100.<br>* Middle blue bar denotes student grade.<br>* Short blue vertical line is the class average for the assignment.<br>* Grade differential to the average seen on right.",
                align='left',
                font_size=8
    )
    
  
    fig.update_layout(height = 300 , margin = {'t':0, 'b':0, 'l':0})


    return fig # You must return all the output(s) in step 2

## Callback for Student id card(s)
@app.callback(
    dash.dependencies.Output('student_card_id','children'),
    [dash.dependencies.Input('pred_table_2', 'active_cell'),
      dash.dependencies.Input('pred_table_2', "page_current"),
      dash.dependencies.Input('pred_table_2', "page_size"),
    ],
    dash.dependencies.State('pred_table_2', 'data')

)
def update_cards(active_cell, page_current, page_size, data):
    id_student = getCellStudentId(active_cell, page_current, page_size, data)
    return f"Student ID:{id_student}" 

## Callback for Student id card
@app.callback(
    dash.dependencies.Output('student_card_id_1','children'),
     [dash.dependencies.Input('pred_table_2', 'active_cell'),
      dash.dependencies.Input('pred_table_2', "page_current"),
      dash.dependencies.Input('pred_table_2', "page_size"),
    ],
    dash.dependencies.State('pred_table_2', 'data')
)

def update_cards(active_cell, page_current, page_size, data):
    id_student = getCellStudentId(active_cell, page_current, page_size, data)
    return f"Student ID:{id_student}" 

## Callback for Student id card
@app.callback(
    
    dash.dependencies.Output('student_card_id_2','children'),
    [dash.dependencies.Input('pred_table_2', 'active_cell'),
      dash.dependencies.Input('pred_table_2', "page_current"),
      dash.dependencies.Input('pred_table_2', "page_size"),
    ],
    dash.dependencies.State('pred_table_2', 'data')
)

def update_cards(active_cell, page_current, page_size, data):
    id_student = getCellStudentId(active_cell, page_current, page_size, data)
    return f"Student ID:{id_student}" 

## Callback for Student id card
@app.callback(
    dash.dependencies.Output('delayed_submission','children'),
    [dash.dependencies.Input('pred_table_2', 'active_cell'),
      dash.dependencies.Input('pred_table_2', "page_current"),
      dash.dependencies.Input('pred_table_2', "page_size"),
    ],
    dash.dependencies.State('pred_table_2', 'data')
)

def update_cards(active_cell, page_current, page_size, data):
    id_student = getCellStudentId(active_cell, page_current, page_size, data)
    delay = qt_60[(qt_60['id_student'] == id_student) & (qt_60['submission_on_time'] == 'Delayed Submission')].shape[0]
    return f"Delayed:{delay}" 

## Callback for Student id card
@app.callback(
    dash.dependencies.Output('not_submitted','children'),
    [dash.dependencies.Input('pred_table_2', 'active_cell'),
      dash.dependencies.Input('pred_table_2', "page_current"),
      dash.dependencies.Input('pred_table_2', "page_size"),
    ],
    dash.dependencies.State('pred_table_2', 'data')
)

def update_cards(active_cell, page_current, page_size, data):
    id_student = getCellStudentId(active_cell, page_current, page_size, data)
    not_submitted = qt_60[(qt_60['id_student'] == id_student) & (qt_60['submitted'] == 'Not Submitted')].shape[0]
    return f"Not Submitted:{not_submitted}" 

@app.callback(
     dash.dependencies.Output('output_display', 'children'),
     [dash.dependencies.Input('pred_table_2', 'active_cell'),
      dash.dependencies.Input('pred_table_2', "page_current"),
      dash.dependencies.Input('pred_table_2', "page_size"),
    ],
     dash.dependencies.State('pred_table_2', 'data')
)
def getActiveCell(active_cell, page_current, page_size, data):
        
    if active_cell:
        col = active_cell['column_id']
        row = active_cell['row']
        cellData = data[page_current * page_size + row][col]
        if col == 'id_student':
            return html.P(["Click in Id student column to make selection", html.Br(), f"Student Id Selected :{cellData}"])
    return html.P(["Click in Id student column to make selection", html.Br(),f"Student Id Selected : {data[0]['id_student']}"])
        

    



## Run the server to create dashboard

In [39]:
app.layout = student

# Run the app
app.run_server(mode="inline", port = 8100)

## 2. Interpret (10 points)

After developing your integrated dashboard, provide your thoughs on the following questions:

2.1 How well will your integrated dashboard help an instructor identify which students to check-in with, and why? (5 points)

To get a sense of which students will pass and who might fail the data contains the predicted probability of failure. This was charted out in the scatter plot above which is the top most chart in the dashboard. This chart has a range slider that can be manipulated from either end. The instructor can focus on the top 10 to 20% of predicted probability of failure and they hover over the individual data points which represent each student and using the dropdown wither select or type in the student id. The scatter plot highlights the selected student and also greys out the students not in the slider range of selection.

- Let's take a look at student id 45664 where the bullet chart shows every score lower than average. This student is highlighted in the bottom left of the top 20% range of students who have a high probability of failing.
- The bullet chart has been richly annotated with explanations to help a first time user interpret the chart in case they have not encountered one like it before.
- Every chart has a tooltip on hover so that individual data points can be readily read out of the chart.
- The student id repeats for every chart whenever the student selection changes.
- Shows if student submitted quiz or assignment in time or not at all.
- The data tables allows for choice of student based on predicted probability of failure shown in diverging color bars at the
  point of divergence = 50%.
- Student selected in the paged table can be examined in learning resources and grades chart below the table and scatter plot.
- The instructor can view the probability of failure column in descending order and get the top few students to select and view details in graphs beneath the table.
  


2.2 Based on the readings and videos, what would you recommend as improvements to your dashboard, and why? (5 points)

- This dashboard has to be made accessible for those whi might have color-blindness,  so the colors will have to be examined.
- There can be audo feedback as well for selections.
- I have created large charts for easy viewing since this is with the notebook environment, but there can be a flow built with more room with charts placed beside each other.
- Although gender, disability and level of educcation are displayed, it might bias the instructor. I included them to make a point that it might be better to not display these or if they are displayed then it is done so with a group rating of performance rather than individual performance.